In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib
import warnings
from sklearn.linear_model import LinearRegression
from sklearn import metrics
warnings.filterwarnings('ignore')

In [78]:
import pandas as pd
import mplfinance as mpf

In [79]:
df = pd.read_csv(r"C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\data\data\btcusdt_1h_train.csv")


In [80]:
def calculate_ichimoku_cloud(data):
    short_window = 19
    long_window = 68
    span_b_window = 168
    displacement = 84
    data['tenkan_sen'] = (data['high'].rolling(window=short_window).max() + data['low'].rolling(window=short_window).min()) / 2
    data['kijun_sen'] = (data['high'].rolling(window=long_window).max() + data['low'].rolling(window=long_window).min()) / 2
    data['senkou_span_a'] = ((data['tenkan_sen'] + data['kijun_sen']) / 2).shift(displacement)
    data['senkou_span_b'] = ((data['high'].rolling(window=span_b_window).max() + data['low'].rolling(window=span_b_window).min()) / 2).shift(displacement)

    # Tenkan Sen and Kijun Sen Cross signals
    data['tenkan_kijun_cross'] = 0
    data.loc[data['tenkan_sen'] > data['kijun_sen'], 'tenkan_kijun_cross'] = 1
    data.loc[data['tenkan_sen'] < data['kijun_sen'], 'tenkan_kijun_cross'] = -1

    # Kumo Signals
    data['above_kumo'] = 0
    data.loc[data['close'] > data[['senkou_span_a', 'senkou_span_b']].max(axis=1), 'above_kumo'] = 1
    data.loc[data['close'] < data[['senkou_span_a', 'senkou_span_b']].min(axis=1), 'above_kumo'] = -1

    # Senkou Span Cross signals
    data['senkou_span_cross'] = 0
    data.loc[data['senkou_span_a'] > data['senkou_span_b'], 'senkou_span_cross'] = 1
    data.loc[data['senkou_span_a'] < data['senkou_span_b'], 'senkou_span_cross'] = -1

    # Kumo Twist signals
    data['kumo_twist'] = 0
    data.loc[(data['senkou_span_a'].shift(1) > data['senkou_span_b'].shift(1)) & (data['senkou_span_a'] <= data['senkou_span_b']), 'kumo_twist'] = 1
    data.loc[(data['senkou_span_a'].shift(1) < data['senkou_span_b'].shift(1)) & (data['senkou_span_a'] >= data['senkou_span_b']), 'kumo_twist'] = -1


    return data

def calculate_obv(data):
    data['daily_return'] = data['close'].pct_change()
    data['obv'] = (data['daily_return'].fillna(0).apply(lambda x: 1 if x > 0 else -1) * data['volume']).cumsum()

    return data

# Example usage:
# Load your OHLCV data into a DataFrame (replace this with your actual data)
ohlc_data = df.copy()
# make the index as datetime
ohlc_data.index = pd.to_datetime(ohlc_data['datetime'])
# drop the datetime column as it is now redundant
ohlc_data.drop('datetime', axis=1, inplace=True)

# Calculate Ichimoku Cloud
ohlc_data = calculate_ichimoku_cloud(ohlc_data)

# Calculate OBV
ohlc_data = calculate_obv(ohlc_data)

# Print the DataFrame with Ichimoku Cloud and OBV
# print(ohlc_data[['open', 'high', 'low', 'close', 'volume', 'tenkan_sen', 'kijun_sen', 'senkou_span_a', 'senkou_span_b', 'obv']])

# # Plot the OHLC with Ichimoku Cloud and OBV
# mpf.plot(ohlc_data, type='candle', mav=(9, 26), volume=True, addplot=[
#     mpf.make_addplot(ohlc_data[['tenkan_sen', 'kijun_sen', 'senkou_span_a', 'senkou_span_b']]),
#     mpf.make_addplot(ohlc_data['obv'], panel=1, color='g', secondary_y=True)
# ])

In [81]:
def calculate_obv_flags(data):
    data['daily_return'] = data['close'].pct_change()
    data['obv'] = (data['daily_return'].fillna(0).apply(lambda x: 1 if x > 0 else -1) * data['volume']).cumsum()

    # Bullish and bearish signals based on OBV
    data['bullish_signal'] = (data['obv'] > data['obv'].shift(1)).astype(int)
    data['bearish_signal'] = (data['obv'] < data['obv'].shift(1)).astype(int)

    # Bullish and bearish divergence signals
    data['bullish_divergence'] = ((data['close'] < data['close'].shift(1)) & (data['obv'] > data['obv'].shift(1))).astype(int)
    data['bearish_divergence'] = ((data['close'] > data['close'].shift(1)) & (data['obv'] < data['obv'].shift(1))).astype(int)

    # Bullish and bearish confirmation signals
    data['bullish_confirmation'] = ((data['close'] > data['close'].shift(1)) & (data['obv'] > data['obv'].shift(1))).astype(int)
    data['bearish_confirmation'] = ((data['close'] < data['close'].shift(1)) & (data['obv'] < data['obv'].shift(1))).astype(int)

    # Create an overall flag column
    data['market_direction'] = 0  # 0 indicates no clear direction
    data.loc[data['bearish_confirmation'] == 1, 'market_direction'] = -1
    data.loc[data['bullish_confirmation'] == 1, 'market_direction'] = 1

    return data

# Example usage:
# Load your OHLCV data into a DataFrame (replace this with your actual data)
calculate_obv_flags(ohlc_data)

,open,high,low,close,volume,tenkan_sen,kijun_sen,senkou_span_a,senkou_span_b,tenkan_kijun_cross,...,kumo_twist,daily_return,obv,bullish_signal,bearish_signal,bullish_divergence,bearish_divergence,bullish_confirmation,bearish_confirmation,market_direction
datetime,,,,,,,,,,,,,,,,,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,NaN,NaN,NaN,NaN,0,...,0,NaN,-443.356199,0,0,0,0,0,0,0
2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,NaN,NaN,NaN,NaN,0,...,0,-0.024093,-827.053205,0,1,0,0,0,1,-1
2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,NaN,NaN,NaN,NaN,0,...,0,0.009628,-397.988633,1,0,0,0,1,0,1
2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,NaN,NaN,NaN,NaN,0,...,0,0.005990,22.098397,1,0,0,0,1,0,1
2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,NaN,NaN,NaN,NaN,0,...,0,0.014242,362.905726,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-24 18:30:00,9521.35,9538.21,9497.44,9503.97,1941.282546,9536.720,9462.5,9167.055,9163.395,1,...,0,-0.001825,-194042.992773,0,1,0,0,0,1,-1
2020-07-24 19:30:00,9503.97,9538.02,9503.04,9537.78,1931.828436,9536.720,9462.5,9167.055,9163.395,1,...,0,0.003557,-192111.164337,1,0,0,0,1,0,1
2020-07-24 20:30:00,9537.77,9566.38,9523.56,9551.68,2154.170510,9536.720,9462.5,9167.055,9163.395,1,...,0,0.001457,-189956.993827,1,0,0,0,1,0,1


In [82]:
ohlc_data = ohlc_data.dropna()
ohlc_data = ohlc_data.reset_index()

In [83]:
ohlc_data.head(10)

,datetime,open,high,low,close,volume,tenkan_sen,kijun_sen,senkou_span_a,senkou_span_b,...,kumo_twist,daily_return,obv,bullish_signal,bearish_signal,bullish_divergence,bearish_divergence,bullish_confirmation,bearish_confirmation,market_direction
0,2018-01-11 17:30:00,13385.10,13942.45,13282.13,13803.00,1048.002392,13184.34,13450.0,16093.2475,14963.12,...,0,0.031221,-425.476089,1,0,0,0,1,0,1
1,2018-01-11 18:30:00,13803.00,13939.00,13628.01,13754.95,992.273766,13184.34,13450.0,16093.2475,14963.12,...,0,-0.003481,-1417.749855,0,1,0,0,0,1,-1
2,2018-01-11 19:30:00,13754.95,14141.00,13680.83,14100.00,1103.789204,13184.34,13450.0,16123.8275,14963.12,...,0,0.025086,-313.960651,1,0,0,0,1,0,1
3,2018-01-11 20:30:00,14070.07,14400.00,13862.00,13958.08,1297.428100,13184.34,13450.0,16096.4875,14963.12,...,0,-0.010065,-1611.388751,0,1,0,0,0,1,-1
4,2018-01-11 21:30:00,13950.05,14190.00,13907.00,13949.26,656.490100,13184.34,13450.0,16232.6400,14963.12,...,0,-0.000632,-2267.878851,0,1,0,0,0,1,-1
5,2018-01-11 22:30:00,13949.26,13966.63,13609.19,13650.01,783.985153,13184.34,13450.0,16227.3000,14963.12,...,0,-0.021453,-3051.864004,0,1,0,0,0,1,-1
6,2018-01-11 23:30:00,13650.01,13849.99,13620.00,13661.50,645.060283,13184.34,13450.0,16194.3225,14963.12,...,0,0.000842,-2406.803721,1,0,0,0,1,0,1
7,2018-01-12 00:30:00,13661.35,13772.52,13205.58,13430.16,703.993324,13012.06,13450.0,16180.0600,14963.12,...,0,-0.016934,-3110.797045,0,1,0,0,0,1,-1
8,2018-01-12 01:30:00,13400.00,13655.92,13050.06,13400.00,852.382387,12952.51,13450.0,16119.5650,14963.12,...,0,-0.002246,-3963.179432,0,1,0,0,0,1,-1
9,2018-01-12 02:30:00,13405.00,13434.56,13008.00,13379.51,915.885084,12900.00,13450.0,15745.6500,14963.12,...,0,-0.001529,-4879.064516,0,1,0,0,0,1,-1


In [84]:
# def combine(df):
#     # Create a new column for the combined indicator
#     df['combined_indicator'] = 0

#     # Define conditions for the combined indicator
#     condition_long = (df['market_direction'] == 1) & (df['above_kumo'] == 1) & (df['senkou_span_cross'] == 1) & (df['kumo_twist'] == 1) & (df['tenkan_kijun_cross'] == 1) # Add more conditions as needed
#     condition_short = (df['market_direction'] == -1) & (df['above_kumo'] == -1) & (df['senkou_span_cross'] == -1) & (df['kumo_twist'] == -1) & (df['tenkan_kijun_cross'] == -1)# Add more conditions as needed

#     # Assign values to the combined indicator based on conditions
#     df.loc[condition_long, 'combined_indicator'] = 1
#     df.loc[condition_short, 'combined_indicator'] = -1

# combine(ohlc_data)

In [85]:
# ohlc_data.to_csv(r"C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\src\logs\obv.csv")

In [86]:
# Select the 5 columns for applying the logic
selected_columns = ['market_direction', 'above_kumo', 'senkou_span_cross', 'kumo_twist', 'tenkan_kijun_cross']

# Define a function to apply the logic
def assign_value(row):
    if row.sum() >= 2:
        return 1
    elif row.sum() <= -2:
        return -1
    else:
        return 0

# Apply the function to create a new column 'final_value'
ohlc_data['flag'] = ohlc_data[selected_columns].apply(assign_value, axis=1)


In [105]:
def print_buckets(df, alphas, rets = ['ret_10', 'ret_30', 'ret_60', 'ret_300', 'tick_10'], aggfunc = ['mean', 'median', 'count'], buckets = 10):
# def print_buckets(df, alphas, rets = ['ret_10', 'ret_30', 'iv_10'], aggfunc = ['mean', 'median', 'count']):
# def print_buckets(df, alphas, rets = ['tick_10', 'tick_30', 'tick_60', 'tick_300'], aggfunc = ['mean', 'median', 'count']):
# def print_buckets(df, alphas, rets = ['ret_60', 'ret_300', 'iv_60'], aggfunc = ['mean', 'median', 'count']):
    for alpha in alphas:
        msg = df.pivot_table(index = pd.qcut(df[alpha],buckets,duplicates='drop'), values=rets, aggfunc=aggfunc)
        print(msg)

def print_corrs(df, alphas, rets = ['ret_10', 'ret_30', 'ret_60', 'ret_300']):
# def print_corrs(df, alphas, rets = ['ret_10', 'ret_30', 'iv_10']):
# def print_corrs(df, alphas, rets = ['tick_10', 'tick_30', 'tick_60', 'tick_300']):
# def print_corrs(df, alphas, rets = ['ret_60', 'ret_300', 'iv_60']):
    msg = "                                      "
    for ret in rets:
        msg += f"{ret:>8s}"
    print(msg)
    for alpha in alphas:
        msg = f"{alpha:30s} corr -> "
        for ret in rets:
            msg += f"{df[alpha].corr(df[ret])*100:7.2f} "
        print(msg)
    print()


In [110]:
def ret(df):
    df['change_5'] = df['close'].rolling(window=5).mean().pct_change() * 1e4

    df['ret_5'] = df['close'].shift(-5)
    df['ret_1'] = df['close'].shift(-1)
    df['ret_10'] = df['close'].shift(-10)
    df['ret_30'] = df['close'].shift(-30)

    df['ret_5'] = (df['ret_5'] - df['close']) / df['close'] * 1e4
    df['ret_1'] = (df['ret_1'] - df['close']) / df['close'] * 1e4
    df['ret_10'] = (df['ret_10'] - df['close']) / df['close'] * 1e4
    df['ret_30'] = (df['ret_30'] - df['close']) / df['close'] * 1e4

    df.head(10)
    df = df.dropna()
    df = df.reset_index(drop=True)
    return df

ohcl_data = ret(ohlc_data)

In [111]:
print_corrs(ohlc_data, ['flag'], ['ret_5', 'ret_1', 'ret_10', 'ret_30'])
print_buckets(ohlc_data, ['flag'], ['ret_5', 'ret_1', 'ret_10', 'ret_30'])

                                         ret_5   ret_1  ret_10  ret_30
flag                           corr ->    1.73    0.05    2.75    3.65 

                   mean                                    median            \
                  ret_1     ret_10     ret_30     ret_5     ret_1    ret_10   
flag                                                                          
(-1.001, 0.0] -0.030300  -3.728395  -9.328533 -1.711250  1.496180  1.648446   
(0.0, 1.0]     0.751803  13.070176  37.255343  6.276982 -0.507775  6.927231   

                                    count                       
                  ret_30     ret_5  ret_1 ret_10 ret_30  ret_5  
flag                                                            
(-1.001, 0.0]   4.801084  2.381812  14997  14997  14985  14997  
(0.0, 1.0]     10.522930  3.621909   7107   7098   7090   7103  


In [87]:
print(ohlc_data['flag'].value_counts())
ohlc_data['signal'] = np.nan

flag
 0    8249
 1    7108
-1    6748
Name: count, dtype: int64


In [88]:
import pandas as pd
import numpy as np

def apply_trading_strategy(df, flag_column, log_column, stop_loss_thresh=-0.07):
    exit_loss = 0
    disable_trading = 0
    compare = 0
    df[log_column] = np.nan

    for i in range(len(df)):
        if df[flag_column].iloc[i] == 1 and compare == 0:
            # No open trade, encounter buy signal
            exit_loss = 0
            buy_price = df["open"].iloc[i]
            compare = 1
            df[log_column].iloc[i] = 1

        elif (df[flag_column].iloc[i] != -1 and compare == 1):
            # Current buy trade, encounter buy signal or no signal - update stop loss
            df[log_column].iloc[i] = 0

            # Calculate pnl, if we exit now
            sell_price = df["open"].iloc[i]
            exit_loss = (sell_price - buy_price) / buy_price

            # Exit trade, if the loss is higher than stop loss
            if exit_loss < stop_loss_thresh and disable_trading == 0:
                df[log_column].iloc[i] = -1
                disable_trading = 1

        elif df[flag_column].iloc[i] == -1 and compare == 1:
            # Current buy trade, encounter sell signal

            # Exit trade
            df[log_column].iloc[i] = -1
            compare = 0
            exit_loss = 0

            # If trade was already exited before, check for disable trading flag here, do nothing here
            if disable_trading == 1:
                disable_trading = 0
                df[log_column].iloc[i] = 0

        # SHORT TRADES
        elif df[flag_column].iloc[i] == -1 and compare == 0:
            # No open trade, encounter sell signal
            exit_loss = 0
            sell_price = df["open"].iloc[i]
            compare = -1
            df[log_column].iloc[i] = -1

        elif (df[flag_column].iloc[i] != 1 and compare == -1):
            # Current sell trade, encounter sell signal or no signal - do nothing, update stop loss
            df[log_column].iloc[i] = 0

            # Calculate pnl, if we exit now
            buy_price = df["open"].iloc[i]
            exit_loss = (sell_price - buy_price) / sell_price

            # Exit trade, if the loss is higher than stop loss
            if exit_loss < stop_loss_thresh and disable_trading == 0:
                df[log_column].iloc[i] = 1
                disable_trading = 1

        elif df[flag_column].iloc[i] == 1 and compare == -1:
            # Exit trade
            df[log_column].iloc[i] = 1
            compare = 0
            exit_loss = 0

            # If trade was already exited before, check for disable trading flag here, do nothing here
            if disable_trading == 1:
                disable_trading = 0
                df[log_column].iloc[i] = 0

        elif df[flag_column].iloc[i] == 0 and compare == 0:
            df[log_column].iloc[i] == 0

    # Fill in any missing values in the new column with 0.
    df[log_column].fillna(0, inplace=True)

    # Close out positions (if needed)
    df[log_column].iloc[-1] = -np.sum(df[log_column])

    # df_filter = df[['datetime', 'open', 'high', 'low', 'close', 'volume', log_column]]
    # df_filter.columns = ["datetime", 'open', 'high', 'low', 'close', 'volume', 'signal']
    # df_filter = df_filter.reset_index(drop=True)
    
    return df

# Example usage:
# Load your DataFrame 'df' with the required columns
# df = ...

# Apply the trading strategy function
result_df = apply_trading_strategy(ohlc_data, flag_column='flag', log_column='signal')
print(sum(abs(result_df["signal"])))
result_df['signal'].value_counts()
# Save the result to a CSV file
# result_df.to_csv('output_file.csv', index=False)


456.0


signal
 0.0    21649
-1.0      228
 1.0      228
Name: count, dtype: int64

In [89]:
# result_df.head()

In [90]:
# ohlc_data.head(20)

In [91]:
# ohlc_data['signal'] = ohlc_data['signal_1']

In [92]:
result_df.to_csv(r"C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\src\logs\obv_stoploss.csv")

In [93]:
# ohlc_data.to_csv(r"C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\src\logs\obv_stoploss.csv")

In [94]:
result_df['signal'].value_counts()

signal
 0.0    21649
-1.0      228
 1.0      228
Name: count, dtype: int64

In [95]:
df_kushal = pd.read_csv(r"C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\src\Ayush\macd_train_1.csv")

In [96]:
df_kushal.head()    

,datetime,open,high,low,close,volume,signal
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,0
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,0
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,0
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,0


In [97]:
df_kushal['signal'].value_counts()  

signal
 0    20492
 1      932
-1      932
Name: count, dtype: int64

In [98]:
# Convert 'datetime' column to datetime64[ns] type
result_df['datetime'] = pd.to_datetime(result_df['datetime'])
df_kushal['datetime'] = pd.to_datetime(df_kushal['datetime'])

# Merge DataFrames on 'datetime' column
df_merged = pd.merge(result_df, df_kushal, on='datetime', how='inner')


In [99]:
df_merged.head()

,datetime,open_x,high_x,low_x,close_x,volume_x,tenkan_sen,kijun_sen,senkou_span_a,senkou_span_b,...,bearish_confirmation,market_direction,flag,signal_x,open_y,high_y,low_y,close_y,volume_y,signal_y
0,2018-01-11 17:30:00,13385.10,13942.45,13282.13,13803.00,1048.002392,13184.34,13450.0,16093.2475,14963.12,...,0,1,0,0.0,13385.10,13942.45,13282.13,13803.00,1048.002392,0
1,2018-01-11 18:30:00,13803.00,13939.00,13628.01,13754.95,992.273766,13184.34,13450.0,16093.2475,14963.12,...,1,-1,-1,-1.0,13803.00,13939.00,13628.01,13754.95,992.273766,0
2,2018-01-11 19:30:00,13754.95,14141.00,13680.83,14100.00,1103.789204,13184.34,13450.0,16123.8275,14963.12,...,0,1,0,0.0,13754.95,14141.00,13680.83,14100.00,1103.789204,1
3,2018-01-11 20:30:00,14070.07,14400.00,13862.00,13958.08,1297.428100,13184.34,13450.0,16096.4875,14963.12,...,1,-1,-1,0.0,14070.07,14400.00,13862.00,13958.08,1297.428100,0
4,2018-01-11 21:30:00,13950.05,14190.00,13907.00,13949.26,656.490100,13184.34,13450.0,16232.6400,14963.12,...,1,-1,-1,0.0,13950.05,14190.00,13907.00,13949.26,656.490100,0


In [100]:
df_merged['signal'] = np.nan
compare = 0
for i in range(len(df_merged)):

    # Opening a trade
    if df_merged['signal_x'].iloc[i] == 1 and df_merged['signal_y'].iloc[i] == 1 and compare == 0:
        df_merged['signal'].iloc[i] = 1
        compare = 1
    elif df_merged['signal_x'].iloc[i] == -1 and df_merged['signal_y'].iloc[i] == -1 and compare == 0:
        df_merged['signal'].iloc[i] = -1
        compare = -1
    elif df_merged['signal_x'].iloc[i] == 1 and df_merged['signal_y'].iloc[i] == 0 and compare == 0:
        df_merged['signal'].iloc[i] = 1
        compare = 1
    elif df_merged['signal_x'].iloc[i] == -1 and df_merged['signal_y'].iloc[i] == 0 and compare == 0:
        compare = -1
        df_merged['signal'].iloc[i] = -1
    elif df_merged['signal_x'].iloc[i] == 0 and df_merged['signal_y'].iloc[i] == 1 and compare == 0:
        df_merged['signal'].iloc[i] = 1
        compare = 1
    elif df_merged['signal_x'].iloc[i] == 0 and df_merged['signal_y'].iloc[i] == -1 and compare == 0:
        compare = -1
        df_merged['signal'].iloc[i] = -1

    # Closing a trade
    elif df_merged['signal_x'].iloc[i] == -1 and df_merged['signal_y'].iloc[i] == 1 and compare == 1:
        df_merged['signal'].iloc[i] = 0
        compare = 1
    elif df_merged['signal_x'].iloc[i] == 1 and df_merged['signal_y'].iloc[i] == -1 and compare == 1:
        df_merged['signal'].iloc[i] = 0
        compare = 1
    elif df_merged['signal_x'].iloc[i] == -1 and df_merged['signal_y'].iloc[i] == 0 and compare == 1:
        df_merged['signal'].iloc[i] = -1
        compare = 0
    elif df_merged['signal_x'].iloc[i] == 1 and df_merged['signal_y'].iloc[i] == 0 and compare == 1:
        df_merged['signal'].iloc[i] = 0
        compare = 1
    elif df_merged['signal_x'].iloc[i] == 0 and df_merged['signal_y'].iloc[i] == 1 and compare == 1:
        df_merged['signal'].iloc[i] = 0
        compare = 1
    elif df_merged['signal_x'].iloc[i] == 0 and df_merged['signal_y'].iloc[i] == -1 and compare == 1:
        df_merged['signal'].iloc[i] = -1
        compare = 0

    elif df_merged['signal_x'].iloc[i] == -1 and df_merged['signal_y'].iloc[i] == 1 and compare == -1:
        df_merged['signal'].iloc[i] = 0
        compare = -1
    elif df_merged['signal_x'].iloc[i] == 1 and df_merged['signal_y'].iloc[i] == -1 and compare == -1:
        df_merged['signal'].iloc[i] = 0
        compare = -1
    elif df_merged['signal_x'].iloc[i] == -1 and df_merged['signal_y'].iloc[i] == 0 and compare == -1:
        df_merged['signal'].iloc[i] = 0
        compare = -1
    elif df_merged['signal_x'].iloc[i] == 1 and df_merged['signal_y'].iloc[i] == 0 and compare == -1:
        df_merged['signal'].iloc[i] = 1
        compare = 0
    elif df_merged['signal_x'].iloc[i] == 0 and df_merged['signal_y'].iloc[i] == 1 and compare == -1:
        df_merged['signal'].iloc[i] = 1
        compare = 0
    elif df_merged['signal_x'].iloc[i] == 0 and df_merged['signal_y'].iloc[i] == -1 and compare == -1:
        df_merged['signal'].iloc[i] = 0
        compare = -1

    

df_merged['signal'] = df_merged['signal'].fillna(0)
df_merged['signal'].value_counts()


signal
 0.0    20238
-1.0      934
 1.0      933
Name: count, dtype: int64

In [101]:
df_merged['close'] = df_merged['close_x']
df_merged['open'] = df_merged['open_x']

In [102]:
df_merged['close'].value_counts()


close
8200.00    7
6700.00    7
6580.00    5
9285.00    5
8350.00    5
          ..
3387.40    1
3372.97    1
3374.89    1
3398.00    1
9557.44    1
Name: count, Length: 21111, dtype: int64

In [103]:
df_merged.to_csv(r"C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\src\logs\ay_kush.csv")

In [104]:
result_df.to_csv(r"C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\src\logs\obv_stoploss.csv")